# Creates the following SBML models needed for the Sub-SBML notebook

1. Transcription SBML model
1. Translation SBML model
1. Signal in mixture model
1. Inducer molecule in abundance in mixture (external)
1. Simple membrane model - One step diffusion
1. Advanced membrane model - Two step diffusion 


## Transcription Model:

Consider the following simple transcription-only model where $G$ is a gene, $T$ is a transcript, and $S$ is the signaling molecule.

We can write the following reduced order dynamics:
1. $G \xrightarrow[]{\rho_{tx}(G, S)} G + T$; 
\begin{align} 
\rho_{tx}(G, S) = G K_{X}\frac{S^{2}}{K_{S}^{2}+S^{2}}
\\
\end{align}
Here, $S$ is the inducer signal that cooperatively activates the transcription of the gene $G$. Since, this is a positive activation of the gene by the inducer, we have a positive proportional Hill function.

1. $T \xrightarrow[]{\delta} \varnothing$; massaction kinetics at rate $\delta$.

In [1]:
try:
    from bioscrape.types import Model
    import numpy as np
    import pylab as plt
    #Create a list of species names (strings)
    species = ["G", "mRNA_T", "S"]

    #create a list of parameters in the form (param_name[string], param_val[number])
    params = [("KX", 2), ("KI", 10), ("n", 2.0), ("delta", .1)]

    #create reaction tuples in the form:
    #(Inputs[string list], Outputs[string list], propensity_type[string], propensity_dict {propensity_param:model_param})
    rxn1 = (["G"], ["G", "mRNA_T"], "proportionalhillpositive", {"d":"G", "s1":"S", "k":"KX", "K":"KI", "n":"n"})

    # Or alternatively, use general propensity (simulation will be slower):
    # rxn2 = (["T"], ["T", "X"], "general", {"rate":"ktl * T/(KR + T)"})

    #Notice that parameters can also take numerical values instead of being named directly
    rxn2 = (["mRNA_T"], [], "massaction", {"k":"delta"})
            
    #Create a list of all reactions
    rxns = [rxn1, rxn2]

    #create an initial condition dictionary species not included in the dictionary will default to 0
    x0 = {"G":1, "S":0, "mRNA_T":0}

    #Instaniate the Model object
    M = Model(species = species, parameters = params, reactions = rxns, initial_condition_dict = x0)
    _ = M.write_sbml_model('transcription_SBML_model.xml')
    print('Model written to a SBML file successfully')
except:
    print('Bioscrape not found')

Model written to a SBML file successfully


## Translation model:

1. $T \xrightarrow[]{\rho_{tl}(T)} T+X$; 
\begin{align} 
\rho_{tl}(T) = K_{TR} \frac{T}{K_{R} + T}
\\
\end{align}
Here $X$ is the protein species.
The lumped parameters $K_{TR}$ and $K_R$ model effects due to ribosome saturation. This is the similar Hill function as derived in the enzymatic reaction example. 

1. $X \xrightarrow[]{\delta} \varnothing$; massaction kinetics at rate $\delta$.


In [2]:
try:
    from bioscrape.types import Model
    import numpy as np
    import pylab as plt
    #Create a list of species names (strings)
    species = ["T", "X"]

    #create a list of parameters in the form (param_name[string], param_val[number])
    params = [("KTR", 5), ("n", 2.0), ("KR", 20), ("delta", .1)]

    #create reaction tuples in the form:
    #(Inputs[string list], Outputs[string list], propensity_type[string], propensity_dict {propensity_param:model_param})
    rxn1 = (["T"], ["T", "X"], "hillpositive", {"s1":"T", "k":"KTR", "K":"KR", "n":1}) 

    #Notice that parameters can also take numerical values instead of being named directly
    rxn2 = (["X"], [], "massaction", {"k":"delta"})
            
    #Create a list of all reactions
    rxns = [rxn1, rxn2]
    x0 = {"X":0,"T":0}
    #Instaniate the Model object
    M = Model(species = species, parameters = params, reactions = rxns, initial_condition_dict = x0)
    _ = M.write_sbml_model('translation_SBML_model.xml')
    print('Model written to a SBML file successfully')
except:
    print('Bioscrape not found')

Model written to a SBML file successfully


## Signal induction model:

1. $\varnothing \xrightarrow[]{\rho(I)} S$; 
\begin{align} 
\rho(S) = K_{0} \frac{I^2}{K_{I} + I^2}
\\
\end{align}
Here $S$ is the signal produced on induction by an inducer $I$.
The lumped parameters $K_{0}$ and $K_S$ model effects of cooperative production of the signal by the inducer. This is the similar Hill function as derived in the enzymatic reaction example. 

In [3]:
try:
    from bioscrape.types import Model
    import numpy as np
    import pylab as plt
    #Create a list of species names (strings)
    species = ["I", "S"]

    #create a list of parameters in the form (param_name[string], param_val[number])
    params = [("KI", 25), ("n", 2.0), ("K0", 20)]

    #create reaction tuples in the form:
    #(Inputs[string list], Outputs[string list], propensity_type[string], propensity_dict {propensity_param:model_param})
    rxn1 = ([], ["S"], "hillpositive", {"s1":"I", "k":"K0", "K":"KI", "n":"n"}) 
            
    #Create a list of all reactions
    rxns = [rxn1]
    x0 = {"I":100, "S":0}
    #Instaniate the Model object
    M = Model(species = species, parameters = params, reactions = rxns,initial_condition_dict = x0)
    _ = M.write_sbml_model('signal_in_mixture.xml')
    print('Model written to a SBML file successfully')
except:
    print('Bioscrape not found')   

Model written to a SBML file successfully


## Inducer reservoir in mixture:

A simple model with only one species - "IPTG" - the inducer. This species is present in abundance in the "external" compartment in this model. Hence, this model represents the exterior of a cell system.

In [5]:
##### Creating a IPTG reservoir model####

from subsbml import createBasicSubsystem, SimpleModel
import libsbml

IPTG_ss, IPTG_model = createBasicSubsystem('reservoir')

per_second = IPTG_model.createNewUnitDefinition(uId = 'per_second',uKind = libsbml.UNIT_KIND_SECOND,
                                                uExponent = -1)
substance = IPTG_model.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, uExponent = 1)


IPTG_model.createNewSpecies('IPTG',sComp = 'reservoir', ListOfAmounts = 1e4, 
                             sConstant = False, sSubstance = 'substance')

# Write to SBML file 
_ = IPTG_ss.writeSBML('IPTG_reservoir.xml')
print('Model written to a SBML file successfully')

Model written to a SBML file successfully


## IPTG simple membrane model

IPTG_internal $\leftrightarrow$ IPTG_external 

In [7]:
##### Creating a basic membrane model for IPTG transport####

from subsbml import createNewSubsystem, SimpleModel, SimpleReaction

IPTG_mb_ss = createNewSubsystem()
model = IPTG_mb_ss.createNewModel('IPTG_membrane','second','mole','substance')

IPTG_mb = SimpleModel(model)

per_second = IPTG_mb.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1)
substance = IPTG_mb.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
IPTG_mb.createNewCompartment('internal','internal',2e-6,'litre',True)
IPTG_mb.createNewCompartment('external','external',1e-6,'litre',True)
##############################################################################################

IPTG_mb.createNewSpecies( 'IPTG','external',0,False,'substance')
IPTG_mb.createNewSpecies( 'IPTG','internal',0,False,'substance')

IPTG_mb.createNewParameter( 'kb',0.01,False,'per_second')
IPTG_mb.createNewParameter( 'kd',0.2,False,'per_second')

# model = IPTG_mb_ss.getSBMLDocument().getModel()
r1_sbml = model.createReaction()
r1 = SimpleReaction(r1_sbml)
r1.setId('r1')
r1.setReversible(True)
r1.createNewReactant('IPTG',False,1)
r1.createNewProduct('IPTG_1', False, 1)
r1.createRate('kb * IPTG - kd * IPTG_1')

# Write to SBML file 
_ = IPTG_mb_ss.writeSBML('membrane_IPTG.xml')
print('Model written to a SBML file successfully')

Model written to a SBML file successfully


## IPTG advanced membrane model

1. IPTG_external + transport_protein $\leftrightarrow$ IPTG_transport_protein 
1. IPTG_transport_protein $\leftrightarrow$ IPTG_internal

In [8]:
##### Creating a advanced membrane model for IPTG transport####

from subsbml import createNewSubsystem, SimpleModel, SimpleReaction

IPTG_mb_ss = createNewSubsystem()
model = IPTG_mb_ss.createNewModel('IPTG_membrane','second','mole','substance')

IPTG_mb = SimpleModel(model)

per_second = IPTG_mb.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1)
substance = IPTG_mb.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
IPTG_mb.createNewCompartment('internal','internal',1e-6,'litre',True)
IPTG_mb.createNewCompartment('external','external',1e-6,'litre',True)
##############################################################################################

IPTG_mb.createNewSpecies( 'IPTG','external',0,False,'substance')
IPTG_mb.createNewSpecies( 'IPTG','internal',0,False,'substance')
IPTG_mb.createNewSpecies( 'transport_protein','internal',1,False,'substance')
IPTG_mb.createNewSpecies( 'IPTG_transport_protein','internal',0,False,'substance')

IPTG_mb.createNewParameter( 'kb',0.01,False,'per_second')
IPTG_mb.createNewParameter( 'kd',0.2,False,'per_second')

model = IPTG_mb_ss.getSBMLDocument().getModel()

r1 = SimpleReaction(model.createReaction())
r1.setId('r1')
r1.setReversible(True)
r1.createNewReactant('IPTG',False,1)
r1.createNewReactant('transport_protein', False, 1)
r1.createNewProduct('IPTG_transport_protein', False, 1)
r1.createRate('kb * IPTG * transport_protein - kd * IPTG_transport_protein')


r2 = SimpleReaction(model.createReaction())
r2.setId('r2')
r2.setReversible(True)
r2.createNewReactant('IPTG_transport_protein',False,1)
r2.createNewProduct('IPTG_1',False,1)
r2.createNewProduct('transport_protein',False,1)
r2.createRate('kd * IPTG_transport_protein - kb * IPTG_1 * transport_protein')


# Write to SBML file 
_ = IPTG_mb_ss.writeSBML('membrane_IPTG_detailed.xml')
print('Model written to a SBML file successfully')

Model written to a SBML file successfully
